## Model Train 1 - Tesis Javier-Uriel

### Importamos algunas librerías que nos serán útiles más adelante

In [1]:
import os
import json
import random
import pandas as pd
import tensorflow as tf
assert (tf.__version__=='2.4.1'), 'Versión incorrecta de Tensorflow, por favor instale 2.4.1'
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

pd.set_option('display.max_columns', None) #Para mostrar todas las columnas
import gc #garbage collector
import gc; gc.enable()

Num GPUs Available:  1


In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


### Leemos el Dataset

In [3]:
#Dataset solo movimientos en Z
rpm_list = ['RPM0', 'RPM1', 'RPM2', 'RPM3']
states_list_org = ["vz","az", "uvz",
                    "p", "q",
                    "wp", "wq", 
                    "ap", "aq"]
dataset_name = "Dataset_Z7_Disturbance_Z"
directory = "../logs/Datasets/"+dataset_name
ORDER = 3
dfs = []
states_list=states_list_org.copy()

In [4]:
for filename in os.listdir(directory):
    if not filename.endswith(".csv"):
        continue
    df = pd.read_csv(os.path.join(directory, filename))
    a = []
    ## Desplazamos estados anteriores        
    for n in range(1,ORDER+1):
        for column in states_list:
            df[column+str(n)] = df[column].shift(periods=n, fill_value=0)
            a.append(column+str(n))
    dfs.append(df)
states_list+=a        
        
dataset = pd.concat(dfs)
dataset.head()

,timestamps,x,y,z,Q1,Q2,Q3,Q4,p,q,r,vx,vy,vz,wp,wq,wr,ax,ay,az,ap,aq,ar,RPM0,RPM1,RPM2,RPM3,ux,uy,uz,uvx,uvy,uvz,up,uq,ur,uwp,uwq,uwr,vz1,az1,uvz1,p1,q1,wp1,wq1,ap1,aq1,vz2,az2,uvz2,p2,q2,wp2,wq2,ap2,aq2,vz3,az3,uvz3,p3,q3,wp3,wq3,ap3,aq3
0,0.000000,0.0,0.0,49.999830,0.0,0.0,0.0,1.0,0.0,-0.0,0.0,0.0,0.0,-0.040833,0.0,0.0,0.0,0.0,0.0,-9.800000,0.0,0.0,0.0,9440.3,9440.3,9440.3,9440.3,0.0,0.0,50.0,0.0,0.0,-0.994789,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.004167,0.0,0.0,49.999562,0.0,0.0,0.0,1.0,0.0,-0.0,0.0,0.0,0.0,-0.064276,0.0,0.0,0.0,0.0,0.0,-5.626198,0.0,0.0,0.0,9440.3,9440.3,9440.3,9440.3,0.0,0.0,50.0,0.0,0.0,-0.994789,0.0,0.0,0.0,0.0,0.0,0.0,-0.040833,-9.800000,-0.994789,0.0,-0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.008333,0.0,0.0,49.999197,0.0,0.0,0.0,1.0,0.0,-0.0,0.0,0.0,0.0,-0.087714,0.0,0.0,0.0,0.0,0.0,-5.625162,0.0,0.0,0.0,9440.3,9440.3,9440.3,9440.3,0.0,0.0,50.0,0.0,0.0,-0.994789,0.0,0.0,0.0,0.0,0.0,0.0,-0.064276,-5.626198,-0.994789,0.0,-0.0,0.0,0.0,0.0,0.0,-0.040833,-9.800000,-0.994789,0.0,-0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.012500,0.0,0.0,49.998733,0.0,0.0,0.0,1.0,0.0,-0.0,0.0,0.0,0.0,-0.111148,0.0,0.0,0.0,0.0,0.0,-5.624082,0.0,0.0,0.0,9440.3,9440.3,9440.3,9440.3,0.0,0.0,50.0,0.0,0.0,-0.994789,0.0,0.0,0.0,0.0,0.0,0.0,-0.087714,-5.625162,-0.994789,0.0,-0.0,0.0,0.0,0.0,0.0,-0.064276,-5.626198,-0.994789,0.0,-0.0,0.0,0.0,0.0,0.0,-0.040833,-9.800000,-0.994789,0.0,-0.0,0.0,0.0,0.0,0.0
4,0.016667,0.0,0.0,49.998173,0.0,0.0,0.0,1.0,0.0,-0.0,0.0,0.0,0.0,-0.134577,0.0,0.0,0.0,0.0,0.0,-5.622958,0.0,0.0,0.0,9440.3,9440.3,9440.3,9440.3,0.0,0.0,50.0,0.0,0.0,-0.994789,0.0,0.0,0.0,0.0,0.0,0.0,-0.111148,-5.624082,-0.994789,0.0,-0.0,0.0,0.0,0.0,0.0,-0.087714,-5.625162,-0.994789,0.0,-0.0,0.0,0.0,0.0,0.0,-0.064276,-5.626198,-0.994789,0.0,-0.0,0.0,0.0,0.0,0.0


### Estados repetidos

En este caso se eliminan estados repetidos y estados que se encuentren en estado transitorio mientras el dron despega o se estabiliza antes de introducir la señal de control.

In [5]:
shape_b4 = dataset.drop(["timestamps"], axis=1).shape
shape_drop= dataset.drop(["timestamps"], axis=1).drop_duplicates().shape
print(f'shape (b4 drop) = {shape_b4}')
print(f'shape = {shape_drop}')
print(f'len (b4 drop) - len = {shape_b4[0]-shape_drop[0]}')

shape (b4 drop) = (4007833, 65)
shape = (3991268, 65)
len (b4 drop) - len = 16565


### División del dataset en estados y acciones

In [6]:
actions = dataset.drop(["timestamps"], axis=1).drop_duplicates()[rpm_list]
print(f'columns = {actions.columns}')
print(f'shape = {actions.shape}')
actions.head()

columns = Index(['RPM0', 'RPM1', 'RPM2', 'RPM3'], dtype='object')
shape = (3991268, 4)


,RPM0,RPM1,RPM2,RPM3
0,9440.3,9440.3,9440.3,9440.3
1,9440.3,9440.3,9440.3,9440.3
2,9440.3,9440.3,9440.3,9440.3
3,9440.3,9440.3,9440.3,9440.3
4,9440.3,9440.3,9440.3,9440.3


#### Normalización de acciones

In [7]:
def normalize_df(df):
    K = df.max().max()
    df_norm = actions/K
    return df_norm, {'K':K}

In [8]:
actions, K = normalize_df(actions)
print(K)
actions.describe()

{'K': 21666.4475}


,RPM0,RPM1,RPM2,RPM3
count,3.991268e+06,3.991268e+06,3.991268e+06,3.991268e+06
mean,6.654759e-01,6.654656e-01,6.654763e-01,6.654851e-01
std,6.111982e-02,6.123138e-02,6.112700e-02,6.102522e-02
min,4.357106e-01,4.357106e-01,4.357106e-01,4.357106e-01
25%,6.658032e-01,6.657335e-01,6.658108e-01,6.657669e-01
50%,6.677804e-01,6.677804e-01,6.677804e-01,6.677804e-01
75%,6.700394e-01,6.700477e-01,6.700425e-01,6.700659e-01
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


#### Definimos los estados

In [9]:
states = dataset.drop(["timestamps"], axis=1).drop_duplicates()[states_list]
print(f'columns = {states.columns}')
print(f'shape = {states.shape}')
states.describe()

columns = Index(['vz', 'az', 'uvz', 'p', 'q', 'wp', 'wq', 'ap', 'aq', 'vz1', 'az1',
       'uvz1', 'p1', 'q1', 'wp1', 'wq1', 'ap1', 'aq1', 'vz2', 'az2', 'uvz2',
       'p2', 'q2', 'wp2', 'wq2', 'ap2', 'aq2', 'vz3', 'az3', 'uvz3', 'p3',
       'q3', 'wp3', 'wq3', 'ap3', 'aq3'],
      dtype='object')
shape = (3991268, 36)


,vz,az,uvz,p,q,wp,wq,ap,aq,vz1,az1,uvz1,p1,q1,wp1,wq1,ap1,aq1,vz2,az2,uvz2,p2,q2,wp2,wq2,ap2,aq2,vz3,az3,uvz3,p3,q3,wp3,wq3,ap3,aq3
count,3.991268e+06,3.991268e+06,3.991268e+06,3.991268e+06,3.991268e+06,3.991268e+06,3.991268e+06,3.991268e+06,3.991268e+06,3.991268e+06,3.991268e+06,3.991268e+06,3.991268e+06,3.991268e+06,3.991268e+06,3.991268e+06,3.991268e+06,3.991268e+06,3.991268e+06,3.991268e+06,3.991268e+06,3.991268e+06,3.991268e+06,3.991268e+06,3.991268e+06,3.991268e+06,3.991268e+06,3.991268e+06,3.991268e+06,3.991268e+06,3.991268e+06,3.991268e+06,3.991268e+06,3.991268e+06,3.991268e+06,3.991268e+06
mean,7.085380e-02,1.105559e-04,5.324777e-02,-2.152827e-04,-2.173570e-04,-4.505836e-05,-1.568587e-04,1.737527e-05,6.190576e-05,7.085265e-02,1.127319e-04,5.324925e-02,-2.150459e-04,-2.171264e-04,-4.513076e-05,-1.571166e-04,8.700425e-07,4.369809e-05,7.085150e-02,1.149082e-04,5.325074e-02,-2.148080e-04,-2.168944e-04,-4.513438e-05,-1.572987e-04,-1.569942e-05,2.553570e-05,7.085037e-02,1.170857e-04,5.325222e-02,-2.145695e-04,-2.166613e-04,-4.506897e-05,-1.574051e-04,-3.233181e-05,7.417210e-06
std,8.894115e-01,1.820634e+00,9.451618e-01,1.862380e-02,1.747403e-02,1.266787e-01,1.220315e-01,1.980234e+00,1.954869e+00,8.893934e-01,1.820624e+00,9.451370e-01,1.862733e-02,1.747754e-02,1.266924e-01,1.220468e-01,1.980403e+00,1.955046e+00,8.893753e-01,1.820614e+00,9.451122e-01,1.863089e-02,1.748109e-02,1.267058e-01,1.220617e-01,1.980571e+00,1.955224e+00,8.893572e-01,1.820603e+00,9.450875e-01,1.863446e-02,1.748467e-02,1.267189e-01,1.220761e-01,1.980740e+00,1.955401e+00
min,-9.426658e+00,-9.800000e+00,-9.640079e+00,-3.536326e-01,-2.834447e-01,-2.441203e+00,-2.568337e+00,-1.762474e+01,-1.727289e+01,-9.426658e+00,-9.800000e+00,-9.640079e+00,-3.536326e-01,-2.834447e-01,-2.441203e+00,-2.568337e+00,-1.762474e+01,-1.727289e+01,-9.426658e+00,-9.800000e+00,-9.640079e+00,-3.536326e-01,-2.834447e-01,-2.441203e+00,-2.568337e+00,-1.762474e+01,-1.727289e+01,-9.426658e+00,-9.800000e+00,-9.640079e+00,-3.536326e-01,-2.834447e-01,-2.441203e+00,-2.568337e+00,-1.762474e+01,-1.727289e+01
25%,-7.424227e-02,-7.223287e-03,-7.338567e-02,-6.184113e-08,-5.221259e-08,-4.008128e-07,-3.313994e-07,-3.759924e-06,-3.126353e-06,-7.422554e-02,-7.221279e-03,-7.337286e-02,-6.184207e-08,-5.221435e-08,-4.008619e-07,-3.314166e-07,-3.761439e-06,-3.127032e-06,-7.421232e-02,-7.219237e-03,-7.337286e-02,-6.184335e-08,-5.221585e-08,-4.008998e-07,-3.314335e-07,-3.762553e-06,-3.127948e-06,-7.419759e-02,-7.216723e-03,-7.336119e-02,-6.184448e-08,-5.221695e-08,-4.009252e-07,-3.314536e-07,-3.763411e-06,-3.128636e-06
50%,7.846921e-17,1.536583e-15,0.000000e+00,6.202457e-19,-0.000000e+00,1.042012e-17,7.132669e-18,-7.998310e-27,-4.188851e-27,7.812540e-17,1.535488e-15,0.000000e+00,6.130119e-19,-0.000000e+00,1.022550e-17,6.957236e-18,-7.846701e-27,-4.094188e-27,7.770564e-17,1.533552e-15,0.000000e+00,6.062379e-19,-0.000000e+00,9.924971e-18,6.957220e-18,-7.697310e-27,-4.016535e-27,7.746774e-17,1.531555e-15,0.000000e+00,6.000400e-19,0.000000e+00,9.830869e-18,6.957199e-18,-7.561232e-27,-3.927788e-27
75%,1.294622e-01,4.167029e-03,1.172527e-01,6.017163e-08,4.970019e-08,4.693793e-07,3.776434e-07,3.454846e-06,2.828179e-06,1.294529e-01,4.166670e-03,1.172383e-01,6.018729e-08,4.971202e-08,4.694050e-07,3.777604e-07,3.455104e-06,2.828163e-06,1.294368e-01,4.166295e-03,1.172383e-01,6.020441e-08,4.972395e-08,4.694651e-07,3.778643e-07,3.455317e-06,2.828103e-06,1.294274e-01,4.165810e-03,1.172383e-01,6.021733e-08,4.974297e-08,4.695572e-07,3.779386e-07,3.455531e-06,2.828011e-06
max,9.720978e+00,1.610805e+01,9.640079e+00,3.149456e-01,2.485051e-01,2.744562e+00,2.167848e+00,1.725882e+01,1.708771e+01,9.720978e+00,1.610805e+01,9.640079e+00,3.149456e-01,2.485051e-01,2.744562e+00,2.167848e+00,1.725882e+01,1.708771e+01,9.720978e+00,1.610805e+01,9.640079e+00,3.149456e-01,2.485051e-01,2.744562e+00,2.167848e+00,1.725882e+01,1.708771e+01,9.720978e+00,1.610805e+01,9.640079e+00,3.149456e-01,2.485051e-01,2.744562e+00,2.167848e+00,1.725882e+01,1

In [10]:
del dataset

### Dividimos el dataset

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(states, actions, test_size=0.05)
ins = states.shape[1]
outs = actions.shape[1]
del states, actions

## Keras Model

#### Early Stopping

In [21]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

#### Definición del Modelo

In [22]:
inputs = tf.keras.Input(shape=(ins,))
x = tf.keras.layers.Dense(2*ins, activation=tf.nn.relu)(inputs)
x = tf.keras.layers.Dense(2*ins, activation=tf.nn.relu)(x)
outputs = tf.keras.layers.Dense(outs, activation=tf.nn.sigmoid)(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

#### Compilado el Modelo

In [23]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

#### Entrenamiento del Modelo

In [24]:
model.fit(X_train, Y_train, validation_split=0.2, epochs=250, batch_size=25000, callbacks=[callback])

Epoch 1/250
122/122 [==============================] - 2s 16ms/step - loss: 0.0133 - mean_squared_error: 0.0133 - val_loss: 5.0960e-04 - val_mean_squared_error: 5.0960e-04
Epoch 2/250
122/122 [==============================] - 2s 13ms/step - loss: 4.2909e-04 - mean_squared_error: 4.2909e-04 - val_loss: 3.1053e-04 - val_mean_squared_error: 3.1053e-04
Epoch 3/250
122/122 [==============================] - 2s 14ms/step - loss: 2.8733e-04 - mean_squared_error: 2.8733e-04 - val_loss: 2.5674e-04 - val_mean_squared_error: 2.5674e-04
Epoch 4/250
122/122 [==============================] - 2s 13ms/step - loss: 2.4832e-04 - mean_squared_error: 2.4832e-04 - val_loss: 2.3254e-04 - val_mean_squared_error: 2.3254e-04
Epoch 5/250
122/122 [==============================] - 2s 13ms/step - loss: 2.2106e-04 - mean_squared_error: 2.2106e-04 - val_loss: 2.1779e-04 - val_mean_squared_error: 2.1779e-04
Epoch 6/250
122/122 [==============================] - 2s 14ms/step - loss: 2.0860e-04 - mean_squared_error:

122/122 [==============================] - 2s 17ms/step - loss: 1.2484e-04 - mean_squared_error: 1.2484e-04 - val_loss: 1.2625e-04 - val_mean_squared_error: 1.2625e-04
Epoch 47/250
122/122 [==============================] - 2s 15ms/step - loss: 1.2363e-04 - mean_squared_error: 1.2363e-04 - val_loss: 1.2578e-04 - val_mean_squared_error: 1.2578e-04
Epoch 48/250
122/122 [==============================] - 2s 15ms/step - loss: 1.2268e-04 - mean_squared_error: 1.2268e-04 - val_loss: 1.2522e-04 - val_mean_squared_error: 1.2522e-04
Epoch 49/250
122/122 [==============================] - 2s 17ms/step - loss: 1.2409e-04 - mean_squared_error: 1.2409e-04 - val_loss: 1.2505e-04 - val_mean_squared_error: 1.2505e-04
Epoch 50/250
122/122 [==============================] - 2s 16ms/step - loss: 1.2312e-04 - mean_squared_error: 1.2312e-04 - val_loss: 1.2466e-04 - val_mean_squared_error: 1.2466e-04
Epoch 51/250
122/122 [==============================] - 2s 15ms/step - loss: 1.2305e-04 - mean_squared_error

122/122 [==============================] - 2s 13ms/step - loss: 1.1128e-04 - mean_squared_error: 1.1128e-04 - val_loss: 1.1476e-04 - val_mean_squared_error: 1.1476e-04
Epoch 92/250
122/122 [==============================] - 2s 14ms/step - loss: 1.1186e-04 - mean_squared_error: 1.1186e-04 - val_loss: 1.1221e-04 - val_mean_squared_error: 1.1221e-04
Epoch 93/250
122/122 [==============================] - 2s 13ms/step - loss: 1.1213e-04 - mean_squared_error: 1.1213e-04 - val_loss: 1.1306e-04 - val_mean_squared_error: 1.1306e-04
Epoch 94/250
122/122 [==============================] - 2s 14ms/step - loss: 1.0998e-04 - mean_squared_error: 1.0998e-04 - val_loss: 1.1358e-04 - val_mean_squared_error: 1.1358e-04
Epoch 95/250
122/122 [==============================] - 2s 13ms/step - loss: 1.1226e-04 - mean_squared_error: 1.1226e-04 - val_loss: 1.1309e-04 - val_mean_squared_error: 1.1309e-04
Epoch 96/250
122/122 [==============================] - 2s 15ms/step - loss: 1.1060e-04 - mean_squared_error

122/122 [==============================] - 2s 17ms/step - loss: 1.0663e-04 - mean_squared_error: 1.0663e-04 - val_loss: 1.1115e-04 - val_mean_squared_error: 1.1115e-04
Epoch 137/250
122/122 [==============================] - 2s 15ms/step - loss: 1.0723e-04 - mean_squared_error: 1.0723e-04 - val_loss: 1.0950e-04 - val_mean_squared_error: 1.0950e-04
Epoch 138/250
122/122 [==============================] - 2s 14ms/step - loss: 1.0614e-04 - mean_squared_error: 1.0614e-04 - val_loss: 1.0905e-04 - val_mean_squared_error: 1.0905e-04
Epoch 139/250
122/122 [==============================] - 2s 15ms/step - loss: 1.0606e-04 - mean_squared_error: 1.0606e-04 - val_loss: 1.0760e-04 - val_mean_squared_error: 1.0760e-04
Epoch 140/250
122/122 [==============================] - 2s 16ms/step - loss: 1.0603e-04 - mean_squared_error: 1.0603e-04 - val_loss: 1.0792e-04 - val_mean_squared_error: 1.0792e-04
Epoch 141/250
122/122 [==============================] - 2s 18ms/step - loss: 1.0580e-04 - mean_squared_

122/122 [==============================] - 2s 14ms/step - loss: 1.0095e-04 - mean_squared_error: 1.0095e-04 - val_loss: 1.0389e-04 - val_mean_squared_error: 1.0389e-04
Epoch 182/250
122/122 [==============================] - 2s 14ms/step - loss: 1.0180e-04 - mean_squared_error: 1.0180e-04 - val_loss: 1.0370e-04 - val_mean_squared_error: 1.0370e-04
Epoch 183/250
122/122 [==============================] - 2s 14ms/step - loss: 1.0108e-04 - mean_squared_error: 1.0108e-04 - val_loss: 1.0401e-04 - val_mean_squared_error: 1.0401e-04
Epoch 184/250
122/122 [==============================] - 2s 14ms/step - loss: 1.0224e-04 - mean_squared_error: 1.0224e-04 - val_loss: 1.0279e-04 - val_mean_squared_error: 1.0279e-04
Epoch 185/250
122/122 [==============================] - 2s 13ms/step - loss: 1.0161e-04 - mean_squared_error: 1.0161e-04 - val_loss: 1.0342e-04 - val_mean_squared_error: 1.0342e-04
Epoch 186/250
122/122 [==============================] - 2s 14ms/step - loss: 1.0094e-04 - mean_squared_

122/122 [==============================] - 2s 15ms/step - loss: 9.8139e-05 - mean_squared_error: 9.8139e-05 - val_loss: 1.0304e-04 - val_mean_squared_error: 1.0304e-04
Epoch 227/250
122/122 [==============================] - 2s 16ms/step - loss: 9.9537e-05 - mean_squared_error: 9.9537e-05 - val_loss: 1.0201e-04 - val_mean_squared_error: 1.0201e-04
Epoch 228/250
122/122 [==============================] - 2s 15ms/step - loss: 9.9160e-05 - mean_squared_error: 9.9160e-05 - val_loss: 1.0020e-04 - val_mean_squared_error: 1.0020e-04
Epoch 229/250
122/122 [==============================] - 2s 16ms/step - loss: 9.8354e-05 - mean_squared_error: 9.8354e-05 - val_loss: 1.0803e-04 - val_mean_squared_error: 1.0803e-04
Epoch 230/250
122/122 [==============================] - 2s 16ms/step - loss: 1.0117e-04 - mean_squared_error: 1.0117e-04 - val_loss: 1.0010e-04 - val_mean_squared_error: 1.0010e-04
Epoch 231/250
122/122 [==============================] - 2s 15ms/step - loss: 9.8639e-05 - mean_squared_

#### Evaluación del Modelo

In [25]:
loss, mean_sq = model.evaluate(X_test, Y_test)
print(f'mean_sq: {mean_sq} -> {mean_sq*K["K"]} RPM')
print(f'loss: {loss} -> {loss*K["K"]} RPM')

6237/6237 [==============================] - 17s 3ms/step - loss: 9.5615e-05 - mean_squared_error: 9.5615e-05
mean_sq: 9.561532351654023e-05 -> 2.071644387166634 RPM
loss: 9.561532351654023e-05 -> 2.071644387166634 RPM


#### Se guarda el Modelo

In [26]:
model.save(f'../Models/{dataset_name}.h5')

In [27]:
model = tf.keras.models.load_model(f'../Models/{dataset_name}_2.h5')
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 36)]              0         
_________________________________________________________________
dense (Dense)                (None, 72)                2664      
_________________________________________________________________
dense_1 (Dense)              (None, 72)                5256      
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 292       
Total params: 8,212
Trainable params: 8,212
Non-trainable params: 0
_________________________________________________________________


In [28]:
%%time
x_test = X_test.sample(n=10, random_state=1)
for index, sample in x_test.iterrows():
    print(model.predict([list(sample)])*K['K'])

[[14515.665 14515.199 14510.113 14520.775]]
[[14465.469 14466.786 14463.589 14465.057]]
[[14468.605 14470.953 14466.574 14473.572]]
[[14465.988 14465.906 14463.074 14465.88 ]]
[[14298.344 14316.94  14308.325 14301.702]]
[[14466.99  14467.572 14465.129 14472.138]]
[[14625.234  14636.0625 14632.525  14629.96  ]]
[[14518.902 14520.231 14531.022 14526.865]]
[[14560.705 14557.883 14532.643 14552.146]]
[[17186.357 17142.918 17247.58  17237.582]]
Wall time: 606 ms


In [29]:
%%time
x_test = [0]*len(states_list)
x_test[0] = 1
x_test[1] = -9.8
x_test[2] = 3
print(model.predict([list(x_test)])*K['K'])

[[18778.92  18985.613 18903.902 19023.064]]
Wall time: 59 ms
